# problema caxeiro viajante aluguel de carros 



### Descrição do problema
 Problema do Caixeiro Viajante (TSP) com aluguel de carros é uma extensão do clássico Problema do Caixeiro Viajante, que envolve encontrar a rota mais curta que visita um conjunto de cidades exatamente uma vez e retorna à cidade de origem. No TSP com aluguel de carros, além de visitar as cidades, o caixeiro também pode alugar carros nas cidades para viajar entre elas. O objetivo é minimizar a distância total percorrida, incluindo o aluguel de carros.

Análise de Complexidade:
O TSP com aluguel de carros é um problema NP-difícil. A complexidade computacional do TSP comum já é alta, sendo NP-completo, e a adição do aluguel de carros aumenta a complexidade do problema. A complexidade exata do TSP com aluguel de carros depende de vários fatores, incluindo o número de cidades, o número de carros disponíveis e as restrições de aluguel.


Exact algorithms can find the optimal solution but are only feasible for small problem instances. Examples of exact algorithms include:

- Brute-force search: This involves trying all possible permutations of paths and selecting the shortest one. However, this method quickly becomes impractical even for small instances of the problem due to its factorial time complexity.
- Dynamic programming: The Held-Karp algorithm is an example of a dynamic programming approach to solve the TSP. It has a running time of O(n^2 * 2^n).
- Quantum exact algorithm: More recently, Ambainis et al. proposed a quantum exact algorithm for the TSP that has a running time of O(1.728^n). This algorithm is currently the best-known algorithm for solving the TSP exactly, but it is not yet practical for solving large instances of the problem.
- Branch-and-bound and branch-and-cut algorithms: These algorithms can handle TSPs with 40-60 cities and larger instances respectively. The Concorde TSP Solver, a branch-and-cut algorithm, can solve a TSP with almost 86,000 cities.

https://www.baeldung.com/cs/tsp-exact-solutions-vs-heuristic-vs-approximation-algorithms

# Instance creator

In [1]:
import random 

def generate_symmetric_matrix(size):
    matrix = [[0 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(i, size):
            if(i!=j):
                value = random.randint(1, 100)  # Random integer between 1 and 100
                matrix[i][j] = value
                matrix[j][i] = value  # Make it symmetric
    return matrix

def generate_cars_instance_file(citys, cars, file_name):
    with open(file_name, 'w') as file:
        file.write(f"NAME : Test{citys}n\n")
        file.write("TYPE : CaRS\n")
        file.write("COMMENT : Instances for the CaRS Problem (Asconavieta & Goldbarg)\n")
        file.write(f"DIMENSION : {citys}\n")
        file.write(f"CARS_NUMBER : {cars}\n")
        file.write("EDGE_WEIGHT_TYPE : EXPLICIT\n")
        file.write("EDGE_WEIGHT_FORMAT : FULL_MATRIX\n")

        file.write("EDGE_WEIGHT_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")


        file.write("RETURN_RATE_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")

        file.write("EOF\n")

# Example usage:
generate_cars_instance_file(6,2, "teste.car")


# Instance reader

In [2]:
class CaRSInstance:
    def __init__(self):
        self.name = ""
        self.type = ""
        self.comment = ""
        self.dimension = 0
        self.cars_number = 0
        self.edge_weight_type = ""
        self.edge_weight_format = ""

class CaRSCar:
    def __init__(self, car_id):
        self.car_id = car_id
        self.distance_matrix = []
        self.return_rate_matrix = []

def read_cars_instance(file_path):
    cars_instance = CaRSInstance()
    cars = []  # List to store Car objects

    with open(file_path, 'r') as file:
        lines = file.readlines()

    section = None
    car_id = None  # To keep track of the current car
    edge_weight_section = False
    return_rate_section = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("EDGE_WEIGHT_SECTION"):
            edge_weight_section = True
            section = "EDGE_WEIGHT"
            continue
        elif line.startswith("RETURN_RATE_SECTION"):
            return_rate_section = True
            edge_weight_section = False
            section = "RETURN_RATE"
            continue
        elif line.startswith("EOF"):
            section = None  # Mark the end of the section
            continue

        if edge_weight_section:
            if line.isdigit():
                car_id = int(line)
                cars.append(CaRSCar(car_id))
            else:
                values = line.split()
                cars[-1].distance_matrix.append([int(value) for value in values])
        elif return_rate_section:
            if line.isdigit():
                car_id = int(line)
            else:
                values = line.split()
                cars[car_id].return_rate_matrix.append([int(value) for value in values])

    for line in lines:
        key_value = line.split(":")
        if len(key_value) == 2:
            key, value = key_value
            key = key.strip()
            value = value.strip()

            if key == "NAME":
                cars_instance.name = value
            elif key == "TYPE":
                cars_instance.type = value
            elif key == "COMMENT":
                cars_instance.comment = value
            elif key == "DIMENSION":
                cars_instance.dimension = int(value)
            elif key == "CARS_NUMBER":
                cars_instance.cars_number = int(value)
            elif key == "EDGE_WEIGHT_TYPE":
                cars_instance.edge_weight_type = value
            elif key == "EDGE_WEIGHT_FORMAT":
                cars_instance.edge_weight_format = value

    return cars_instance, cars

# Usage example
file_path = "teste.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/simple.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/Bolivia10n.car"  # Replace with the path to your input file
cars_instance, cars = read_cars_instance(file_path)

# Access the parsed data
print("Name:", cars_instance.name)
print("Type:", cars_instance.type)
print("Comment:", cars_instance.comment)
print("Dimension:", cars_instance.dimension)
print("Cars Number:", cars_instance.cars_number)
print("Edge Weight Type:", cars_instance.edge_weight_type)
print("Edge Weight Format:", cars_instance.edge_weight_format)


Name: Test6n
Type: CaRS
Comment: Instances for the CaRS Problem (Asconavieta & Goldbarg)
Dimension: 6
Cars Number: 2
Edge Weight Type: EXPLICIT
Edge Weight Format: FULL_MATRIX


In [3]:

# Access the distance matrices for each car
for car in cars:
    print(f"Car {car.car_id} Distance Matrix:")
    for row in car.distance_matrix:
        print(row)

# Access the return rate matrices for each car
for car in cars:
    print(f"Car {car.car_id} Return Rate Matrix:")
    for row in car.return_rate_matrix:
        print(row)

Car 0 Distance Matrix:
[0, 20, 55, 51, 31, 19]
[20, 0, 44, 71, 11, 2]
[55, 44, 0, 58, 5, 91]
[51, 71, 58, 0, 72, 51]
[31, 11, 5, 72, 0, 59]
[19, 2, 91, 51, 59, 0]
Car 1 Distance Matrix:
[0, 81, 80, 53, 12, 5]
[81, 0, 29, 93, 37, 4]
[80, 29, 0, 91, 26, 79]
[53, 93, 91, 0, 90, 50]
[12, 37, 26, 90, 0, 77]
[5, 4, 79, 50, 77, 0]
Car 0 Return Rate Matrix:
[0, 39, 70, 39, 59, 26]
[39, 0, 74, 32, 53, 30]
[70, 74, 0, 47, 39, 30]
[39, 32, 47, 0, 40, 38]
[59, 53, 39, 40, 0, 28]
[26, 30, 30, 38, 28, 0]
Car 1 Return Rate Matrix:
[0, 94, 57, 9, 92, 96]
[94, 0, 80, 33, 17, 56]
[57, 80, 0, 37, 75, 93]
[9, 33, 37, 0, 8, 33]
[92, 17, 75, 8, 0, 8]
[96, 56, 93, 33, 8, 0]


# simulated anealing 

This code implements the Simulated Annealing algorithm. Simulated Annealing is a metaheuristic algorithm used to solve optimization problems. It is inspired by the annealing process in metallurgy, where a material is heated and then slowly cooled to reduce defects and improve its structure.

The `simulated_annealing` function takes several parameters: `num_iterations`, `initial_temperature`, `cooling_rate`, `route`, `car_assignments`, and `cars`. 

- `num_iterations` specifies the number of iterations the algorithm will run for.
- `initial_temperature` is the initial temperature of the system. It controls the probability of accepting worse solutions.
- `cooling_rate` determines how quickly the temperature decreases over time.
- `route` is a list representing the current route.
- `car_assignments` is a list representing the current assignments of cars to customers.
- `cars` is a list of available cars.

The function initializes the current cost by calling the `calculate_total_distance` function, passing in the current `route`, `car_assignments`, and `cars`. It also initializes the `best_route`, `best_car_assignments`, and `best_cost` variables with the current values.

The algorithm then enters a loop that runs for `num_iterations` iterations. In each iteration, it calculates the current temperature based on the initial temperature and the cooling rate. It then generates a random neighbor solution by calling the `generate_random_neighbor` function, passing in the current `route` and `car_assignments`. The `generate_random_neighbor` function returns a new route and car assignments.

The algorithm calculates the cost of the neighbor solution by calling the `calculate_total_distance` function with the neighbor route, neighbor car assignments, and cars. It then checks if the neighbor solution should be accepted as the new current solution. This is determined by the `accept_solution` function, which compares the current cost and neighbor cost with the temperature. If the neighbor solution is accepted, the current route, car assignments, and cost are updated.

The algorithm also checks if the current cost is better than the best cost found so far. If it is, the best route, car assignments, and cost are updated.

After the loop finishes, the function returns the best route, car assignments, and cost.

Overall, the code implements the Simulated Annealing algorithm to find the best route and car assignments for a given problem. It iteratively explores neighbor solutions and accepts worse solutions with a certain probability based on the temperature. The algorithm gradually decreases the temperature over time, allowing it to escape local optima and potentially find better solutions.

In [4]:
import random
import math

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0
    route = [0] + route + [0]
    current_car = car_assignments[0]
    current_location = 0 
    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance

# Função para gerar um vizinho aleatório
def generate_random_neighbor(route, car_assignments):
    new_route = route[:]
    new_car_assignments = car_assignments[:]  

    num_swaps = random.randint(1, len(route) // 2)

    for _ in range(num_swaps):
        city1, city2 = random.sample(range(1, len(route) - 1), 2)
        new_route[city1], new_route[city2] = new_route[city2], new_route[city1]

        city1, city2 = random.sample(range(1, len(route) - 1), 2)
        car1, car2 = new_car_assignments[city1], new_car_assignments[city2]
        new_car_assignments[city1], new_car_assignments[city2] = car2, car1
    return new_route, new_car_assignments

# Função de aceitação com probabilidade
def accept_solution(current_cost, neighbor_cost, temperature):
    if neighbor_cost < current_cost:
        return True
    else:
        probability = math.exp(-(neighbor_cost - current_cost) / temperature)
        return random.random() < probability

# Simulated Annealing
def simulated_annealing(num_iterations, initial_temperature, cooling_rate, route, car_assignments, cars):
    current_cost = calculate_total_distance(route, car_assignments, cars)
    best_route = route
    best_car_assignments = car_assignments
    best_cost = current_cost

    for i in range(num_iterations):
        temperature = initial_temperature * math.exp(-cooling_rate * i)
        neighbor_route, neighbor_car_assignments = generate_random_neighbor(route, car_assignments)
        neighbor_cost = calculate_total_distance(neighbor_route, neighbor_car_assignments, cars)
        if accept_solution(current_cost, neighbor_cost, temperature):
            route, car_assignments = neighbor_route, neighbor_car_assignments
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_route, best_car_assignments, best_cost = route, car_assignments, current_cost

    return best_route, best_car_assignments, best_cost

# Parâmetros de exemplo
num_cities = cars_instance.dimension
num_cars = cars_instance.cars_number
initial_car_assignments = [random.randint(0, num_cars - 1) for _ in range(0, num_cities)]    
initial_route = random.sample(range(1, num_cities), num_cities-1)
num_iterations = num_cities * 10
initial_temperature = 100.0
cooling_rate = 0.03

# Executar Simulated Annealing
best_route, best_car_assignments, best_cost = simulated_annealing(num_iterations, initial_temperature, cooling_rate, initial_route, initial_car_assignments, cars)

print("Melhor rota:", best_route)
print("Melhor alocação de carros:", best_car_assignments)
print("Melhor custo:", best_cost)


Melhor rota: [3, 2, 4, 5, 1]
Melhor alocação de carros: [0, 1, 0, 0, 0, 0]
Melhor custo: 225


# Tabu search

This code implements the Tabu Search algorithm to solve a routing problem. The goal is to find the best route and car assignments that minimize the total distance traveled.

The `generate_neighbors` function takes a route and car assignments as input and generates a list of neighboring solutions. It does this by calling the `generate_random_neighbor` function, which generates a random neighbor by making a small modification to the current solution. The new route and car assignments are added to the list of neighbors.

The `tabu_search` function is the main function that performs the Tabu Search algorithm. It takes several parameters: `num_iterations` specifies the number of iterations to run the algorithm, `tabu_list_size` specifies the size of the tabu list, `initial_route` and `initial_car_assignments` specify the initial solution, and `cars` is a list of available cars.

The function initializes the current solution, best solution, and best cost variables with the initial solution. It also initializes an empty tabu list.

The algorithm then enters a loop that runs for the specified number of iterations. In each iteration, it generates a list of neighbors by calling the `generate_neighbors` function with the current solution and car assignments.

Next, it iterates over each neighbor and calculates the cost of the neighbor solution using the `calculate_total_distance` function. It checks if the neighbor cost is better than the current best neighbor cost and if the neighbor solution is not in the tabu list. If both conditions are met, the neighbor solution becomes the new best neighbor.

After iterating over all neighbors, the algorithm checks if a valid best neighbor was found. If not, it breaks out of the loop.

If a valid best neighbor was found, it adds the best neighbor to the tabu list. If the tabu list exceeds the specified size, it removes the oldest entry.

The current solution and cost are updated with the best neighbor solution and cost.

Finally, if the current cost is better than the best cost, the best solution, car assignments, and cost are updated with the current solution, car assignments, and cost.

The algorithm continues to iterate until the specified number of iterations is reached or a valid best neighbor cannot be found.

At the end of the algorithm, the best solution, car assignments, and cost are returned. These represent the solution found by the Tabu Search algorithm.

In [5]:

def generate_neighbors(route, car_assignments):
    neighbors = []
    for i in range(len(route)):
        new_route, new_car_assignments = generate_random_neighbor(route, car_assignments)
        neighbors.append((new_route, new_car_assignments))
    return neighbors

def tabu_search(num_iterations, tabu_list_size, initial_route, initial_car_assignments, cars):
    current_route = initial_route
    current_car_assignments = initial_car_assignments
    best_route = current_route
    best_car_assignments = current_car_assignments
    best_cost = calculate_total_distance(current_route, current_car_assignments, cars)

    tabu_list = []

    for iteration in range(num_iterations):
        neighbors = generate_neighbors(current_route, current_car_assignments)

        # Escolher o melhor vizinho que não está na lista tabu
        best_neighbor = None
        best_neighbor_cost = float('inf')
        
        for neighbor_route, neighbor_car_assignments in neighbors:
            neighbor_cost = calculate_total_distance(neighbor_route, neighbor_car_assignments, cars)
            if neighbor_cost < best_neighbor_cost and (neighbor_route, neighbor_car_assignments) not in tabu_list:
                best_neighbor = (neighbor_route, neighbor_car_assignments)
                best_neighbor_cost = neighbor_cost

        if best_neighbor is None:
            break  # Não foi possível encontrar um vizinho válido

        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Remover o mais antigo da lista tabu

        current_route, current_car_assignments = best_neighbor
        current_cost = best_neighbor_cost

        if current_cost < best_cost:
            best_route = current_route
            best_car_assignments = current_car_assignments
            best_cost = current_cost

    return best_route, best_car_assignments, best_cost

num_cities = cars_instance.dimension
num_cars = cars_instance.cars_number
initial_car_assignments = [random.randint(0, num_cars - 1) for _ in range(0, num_cities)]    
initial_route = random.sample(range(1, num_cities), num_cities-1)
num_iterations = 100
tabu_list_size = 10


best_route, best_car_assignments, best_cost = tabu_search(num_iterations, tabu_list_size, initial_route, initial_car_assignments, cars)

print("Melhor rota:", best_route)
print("Melhor alocação de carros:", best_car_assignments)
print("Melhor custo:", best_cost)

Melhor rota: [1, 2, 4, 3, 5]
Melhor alocação de carros: [1, 1, 1, 0, 1, 1]
Melhor custo: 289


## Algoritmo genetico

The `generate_random_individual` function generates a random individual for the population. It takes two parameters: `num_cities` and `num_cars`. It starts by initializing the individual with a single element, 0. Then, it appends random integers between 0 and `num_cars - 1` to the individual for the remaining `num_cities - 1` elements. Finally, it returns the individual as a list of two lists: the first list represents the order of cities to visit (excluding the starting and ending city), and the second list represents the assignment of cars to each city.

The `initialize_population` function generates a population of random individuals. It takes three parameters: `pop_size`, `num_cities`, and `num_cars`. It uses a list comprehension to create a list of `pop_size` random individuals by calling the `generate_random_individual` function. Finally, it returns the population as a list of individuals.

The `evaluate_population` function evaluates the fitness of each individual in the population. It takes two parameters: `population` and `cars`. It initializes an empty list called `fitness_scores`. Then, it iterates over each individual in the population and calculates the total distance of the route by calling the `calculate_total_distance` function. The fitness score is calculated as the inverse of the total distance. The fitness score is then appended to the `fitness_scores` list. Finally, it returns the `fitness_scores` list.

The `select_parents` function selects two parents from the population based on their fitness scores. It takes two parameters: `population` and `fitness_scores`. It calculates the total fitness by summing up all the fitness scores. Then, it calculates the probabilities of selecting each individual as a parent by dividing each fitness score by the total fitness. The `random.choices` function is used to select two parents from the population based on the calculated probabilities. Finally, it returns the selected parents as a list.

The `crossover_route` function performs crossover between two routes. It takes three parameters: `list1`, `list2`, and `crossover_point`. It splits `list1` and `list2` at the `crossover_point` and creates two new lists: `left_part1` and `left_part2`. Then, it creates `result1` by concatenating `left_part1` with the elements from `list2` that are not already in `left_part1`. Similarly, it creates `result2` by concatenating `left_part2` with the elements from `list1` that are not already in `left_part2`. Finally, it returns `result1` and `result2`.

The `crossover` function performs crossover between two parents. It takes two parameters: `parent1` and `parent2`. It generates a random crossover point between 1 and the length of `parent1` - 1. Then, it calls the `crossover_route` function to perform crossover on the first list of each parent. The resulting lists are stored in `result1` and `result2`. The second list of each parent is also crossed over at the same crossover point. Finally, it returns two children as a list of two individuals.

The `mutate` function performs mutation on an individual. It takes three parameters: `individual`, `mutation_rate`, and `num_cars`. It iterates over the second list of the individual (representing the assignment of cars to cities) starting from index 1. For each element, it checks if a random number between 0 and 1 is less than the mutation rate. If it is, it assigns a random integer between 0 and `num_cars - 1` to the element. Finally, it returns the mutated individual.

Overall, these functions are used in a genetic algorithm to generate an initial population, evaluate the fitness of each individual, select parents for reproduction, perform crossover and mutation, and create a new population for the next generation.

In [6]:
import random

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0

    current_car = car_assignments[0]
    current_location = 0 
    
    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance
def generate_random_individual(num_cities, num_cars):
    individual0 = [0] 
    for _ in range(0, num_cities - 1):
        individual0.append(random.randint(0, num_cars - 1))
    return [random.sample(range(1, num_cities), num_cities-1), individual0]

def initialize_population(pop_size, num_cities, num_cars):
    population = [generate_random_individual(num_cities, num_cars) for _ in range(pop_size)]
    return population

def evaluate_population(population, cars):
    fitness_scores = []
    for individual in population:
        total_distance = calculate_total_distance([0] + individual[0] + [0], individual[1], cars)
        fitness_scores.append(1 / total_distance)  
    return fitness_scores
def select_parents(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    parents = random.choices(population, probabilities, k=2)
    return parents
def crossover_route(list1, list2, crossover_point):
    left_part1 = list1[:crossover_point]
    left_part2 = list2[:crossover_point]
    result1 = left_part1 + [x for x in list2 if x not in left_part1]
    result2 = left_part2 + [x for x in list1 if x not in left_part2]
    return result1, result2
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    result1, result2 = crossover_route(parent1[0], parent2[0], crossover_point)
    child1 = [result1, parent1[1][:crossover_point] + parent2[1][crossover_point:]]
    child2 = [result2, parent2[1][:crossover_point] + parent1[1][crossover_point:]]
    return child1, child2
def mutate(individual, mutation_rate, num_cars):
    for i in range(1, len(individual)):
        if random.random() < mutation_rate:
            individual[1][i] = random.randint(0, num_cars - 1)
    return individual


This code implements a genetic algorithm to solve a problem related to cars and cities. 

1. The function `genetic_algorithm` takes several parameters: `num_generations` (the number of generations to run the algorithm), `pop_size` (the size of the population), `mutation_rate` (the probability of mutation), `num_cities` (the number of cities in the problem), `num_cars` (the number of cars available), and `cars` (a list of cars with their respective capacities).

2. The function starts by initializing the population using the `initialize_population` function. This function creates a random population of individuals, where each individual represents a possible solution to the problem. Each individual is a list of cities, and each city is represented by an index.

3. The `best_individual_list` is initialized as an empty list. This list will store the best individual found in each generation.

4. The algorithm then enters a loop that iterates over the specified number of generations. In each generation, the following steps are performed:

   a. The fitness scores of the population are evaluated using the `evaluate_population` function. This function calculates the fitness score for each individual in the population based on their total distance traveled.

   b. A new population is created by selecting parents from the current population, performing crossover and mutation operations, and adding the resulting children to the new population. The `select_parents` function is used to select two parents based on their fitness scores. The `crossover` function is used to create two children by combining the genetic material of the parents. The `mutate` function is used to introduce random changes (mutations) in the children's genetic material.

   c. The new population is added to the `new_population` list.

   d. The best individual in the current generation is determined by finding the individual with the highest fitness score. This is done using the `max` function with a lambda function as the key. The lambda function calculates the fitness score for each individual by calling the `calculate_total_distance` function.

   e. The best individual in the current generation is added to the `best_individual_list`.

5. After all the generations have been processed, the algorithm finds the best solution in the `best_individual_list`. This is done by finding the individual with the highest fitness score, again using the `max` function with a lambda function as the key.

6. The total distance of the best solution is calculated by calling the `calculate_total_distance` function with the best individual's cities and cars.

7. Finally, the best individual and its distance are returned as the result of the `genetic_algorithm` function.


In [7]:

def genetic_algorithm(num_generations, pop_size, mutation_rate, num_cities, num_cars, cars):
    population = initialize_population(pop_size, num_cities, num_cars)
    best_individual_list = []

    for generation in range(num_generations):
        fitness_scores = evaluate_population(population, cars)
        new_population = []
        
        for _ in range(pop_size // 2):
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, num_cars)
            child2 = mutate(child2, mutation_rate, num_cars)
            new_population.extend([child1, child2])
        
        best_individual_generation = max(population, key=lambda ind: 1 / calculate_total_distance([0] + ind[0] + [0],ind[1], cars))
        best_individual_list.append(best_individual_generation)
    
    # Após as iterações, encontre a melhor solução na população
    best_individual = max(best_individual_list, key=lambda ind: 1 / calculate_total_distance([0] + ind[0] + [0],ind[1], cars))
    best_distance = calculate_total_distance([0] + best_individual[0] + [0],best_individual[1],  cars)
    
    return best_individual, best_distance

# Executando o algoritmo genético
best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.1, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)

print("Melhor rota:", best_solution)
print("Melhor distância:", best_distance)


Melhor rota: [[4, 2, 3, 5, 1], [0, 0, 0, 0, 0, 0]]
Melhor distância: 167


# algoritmo evolutivo 

In [8]:

# Executando o algoritmo evolutivo
best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.9, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)

print("Melhor rota:", best_solution)
print("Melhor distância:", best_distance)


Melhor rota: [[4, 2, 3, 1, 5], [0, 0, 0, 0, 0, 0]]
Melhor distância: 186


## Backtracking

This code is a solution to the Traveling Salesman Problem (TSP) with the addition of multiple cars. The goal is to find the shortest route that visits all locations and returns to the starting location, while also assigning each location to a specific car.

The code begins by importing the necessary modules: `itertools` for generating permutations and combinations, and `time` for measuring the execution time.

Next, there are two main functions defined: `calculate_total_distance` and `backtrack_tsp_with_cars`.

The `calculate_total_distance` function takes in three parameters: `route`, `car_assignments`, and `cars`. It initializes a variable `total_distance` to 0 and sets the current car to the first car assignment. It also sets the current location to 0, which is typically the starting city.

The function then iterates through each next location in the given route. If the current car is different from the car assigned to the next location, it adds the distance between the current location and the next location, as well as the return rate from the next location to the current location, to the total distance. Otherwise, it only adds the distance between the current location and the next location.

After calculating the distance for each location, the current location is updated to the next location.

Finally, the function returns the total distance.

The `backtrack_tsp_with_cars` function takes in two parameters: `instance` and `cars`. It first retrieves the number of cars and the number of locations from the given instance.

Next, it initializes variables `best_route` and `best_distance` to None and infinity, respectively.

The function then uses nested loops to generate all possible permutations of the locations (excluding the starting location) and all possible combinations of car assignments for each location. For each combination, it creates a full route by adding the starting location (0) at the beginning and end of the route.

For each generated route and car assignment combination, it calls the `calculate_total_distance` function to calculate the total distance.

If the calculated distance is less than the current best distance, it updates the best distance, best route, and best car assignments.

After iterating through all possible combinations, the function returns the best route, best distance, and best car assignments.

The code then measures the execution time by recording the start time before calling the `backtrack_tsp_with_cars` function and the end time after the function finishes. It calculates the elapsed time by subtracting the start time from the end time.

Finally, it prints the execution time, best route, best distance, and best car assignments.

In [9]:
import itertools
import time

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance

def backtrack_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance(full_route, car_assignments, cars)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = backtrack_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 0.007663249969482422 segundos
Melhor rota: (0, 3, 2, 4, 1, 5, 0)
Melhor distância: 146
best_car_assignments: (0, 0, 0, 0, 0, 0)


This code is an algorithm to solve the Traveling Salesman Problem (TSP) with the addition of cars. The goal is to find the shortest route that visits all locations and returns to the starting location, while also assigning each location to a car.

The code begins by importing the necessary modules: `itertools` for generating permutations and combinations, and `time` for measuring the execution time.

Next, there is a function called `calculate_total_distance` that takes in the current route, car assignments, cars, and the best distance found so far. This function calculates the total distance traveled for a given route and car assignments. It iterates over each location in the route and checks if the current car assignment is different from the next location's car assignment. If they are different, it adds the distance from the current location to the next location and the return rate from the next location to the current location. If they are the same, it only adds the distance between the current and next locations. After each iteration, it checks if the current total distance is greater than the best distance found so far. If it is, it immediately returns the current total distance. Finally, it updates the current location to be the next location.

The main function is called `branch_and_bound_tsp_with_cars` and takes in an instance of the problem and a list of cars. It initializes some variables: `num_cars` to store the number of cars, `num_locations` to store the number of locations, `best_route` to store the best route found so far, and `best_distance` to store the best distance found so far (initialized as infinity).

The function then enters a nested loop using `itertools.permutations` and `itertools.product` to generate all possible routes and car assignments. It ensures that the route starts and ends at location 0 by adding it to the beginning and end of the route. For each combination of route and car assignments, it calls the `calculate_total_distance` function to calculate the total distance traveled. If the calculated distance is less than the best distance found so far, it updates the best distance, best route, and best car assignments.

After the nested loop, the function returns the best route, best distance, and best car assignments.

The code then measures the execution time by recording the start time using `time.time()`. It calls the `branch_and_bound_tsp_with_cars` function with the given instance and cars, and stores the returned values in `best_route`, `best_distance`, and `best_car_assignments`. It records the end time and calculates the execution time by subtracting the start time from the end time.

Finally, it prints the execution time, best route, best distance, and best car assignments.

In [10]:
import itertools
import time

def calculate_total_distance(route, car_assignments, cars, best_distance):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]
        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
        if best_distance < total_distance:
            return total_distance

        current_location = next_location

    return total_distance

def branch_and_bound_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance(full_route, car_assignments, cars, best_distance)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = branch_and_bound_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 0.0056667327880859375 segundos
Melhor rota: (0, 3, 2, 4, 1, 5, 0)
Melhor distância: 146
best_car_assignments: (0, 0, 0, 0, 0, 0)
